## Visualize image-specific class saliency with backpropagation

---

The gradients obtained can be used to visualise an image-specific class saliency map, which can gives some intuition on regions within the input image that contribute the most (and least) to the corresponding output.

More details on saliency maps: [Deep Inside Convolutional Networks: Visualising Image Classification Models and Saliency Maps](https://arxiv.org/pdf/1312.6034.pdf).

### 0. Set up (Colab only)

In [1]:
# # Install flashtorch if you don't have it

# !pip install webdataset


# # This mounts your Google Drive to th?e Colab VM.
# from google.colab import drive
# drive.mount('/content/drive')

# # TODO: Enter the foldername in your Drive where you have saved the unzipped
# # assignment folder, e.g. 'cs231n/assignments/assignment3/'
# FOLDERNAME = 'Courses/Spring 2021/CS 231N/cs231n_final_proj/netviz'
# assert FOLDERNAME is not None, "[!] Enter the foldername."

# # Now that we've mounted your Drive, this ensures that
# # the Python interpreter of the Colab VM can load
# # python files from within it.
# import sys
# sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME))

# os.chdir('/content/drive/My Drive/{}'.format(FOLDERNAME+'/notebooks'))

In [31]:
# %matplotlib inline
%config InlineBackend.figure_format = 'retina'

import os
import gc
import sys
import json
import webdataset as wds

sys.path.append('..')

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from tqdm import tqdm
import psutil


from model.baseline_3d_cnn import *
from model.selfattn_3d_cnn import *
from model.resattn_3d_cnn import *
from utils.saliency_map import *
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
data_dir = '../data'
shards_dir = os.path.join(data_dir, 'shards_new')

# Opening JSON file
with open('../parameters.json') as json_file:
    parameters = json.load(json_file)

batch_size = parameters['batch_size']
shard_size = parameters['shard_size']
parameters

{'batch_size': 4, 'shard_size': 16}

In [4]:
urls = [os.path.join(shards_dir, it) for it in os.listdir(shards_dir) if it.endswith('.tar')]
print(urls)
wds_len = len(urls)*shard_size//batch_size

image_dataset = (
    wds
    .WebDataset(urls, length=wds_len)
    .shuffle(shard_size)
    .decode('torch')
    .to_tuple('volumes.pyd', 'labels.pyd', 'studynames.pyd')
    .batched(batch_size)
#     .map_tuple(pre_transforms, identity, identity)
)


image_loader = torch.utils.data.DataLoader(image_dataset, num_workers=0, batch_size=None)

['../data/shards_new/shard-000076.tar', '../data/shards_new/shard-000003.tar', '../data/shards_new/shard-000081.tar', '../data/shards_new/shard-000065.tar', '../data/shards_new/shard-000051.tar', '../data/shards_new/shard-000044.tar', '../data/shards_new/shard-000037.tar', '../data/shards_new/shard-000095.tar', '../data/shards_new/shard-000084.tar', '../data/shards_new/shard-000090.tar', '../data/shards_new/shard-000040.tar', '../data/shards_new/shard-000062.tar', '../data/shards_new/shard-000022.tar', '../data/shards_new/shard-000078.tar', '../data/shards_new/shard-000045.tar', '../data/shards_new/shard-000023.tar', '../data/shards_new/shard-000058.tar', '../data/shards_new/shard-000063.tar', '../data/shards_new/shard-000086.tar', '../data/shards_new/shard-000071.tar', '../data/shards_new/shard-000067.tar', '../data/shards_new/shard-000013.tar', '../data/shards_new/shard-000034.tar', '../data/shards_new/shard-000072.tar', '../data/shards_new/shard-000082.tar', '../data/shards_new/shar

### 1. Load an image 

In [5]:
# patient_num = 1

# for t, (x, y, z) in enumerate(image_loader):
#     if t > 0:
#         break
#     img_df = x[patient_num, :, :, :].detach().numpy()
#     img_y = y[patient_num, :]
# del x # For now for memory reasons

In [6]:
# fig, axs = plt.subplots(5,8, figsize=(15, 6))
# axs = axs.ravel()
# for i in range(40):
#     axs[i].imshow(img_df[i,:,:])

### 2. Load a pre-trained Model

In [61]:
ckpt_dir = os.path.join('..', 'runs', 'baseline')
ckpt_path = os.path.join(ckpt_dir, 'baseline_final_model.pt') # COLAB
ckpt = torch.load(ckpt_path)

ckpt_model = baseline_3DCNN(in_num_ch=1)
ckpt_model.load_state_dict(ckpt)

plot_dir = os.path.join('..', 'net_visualization', 'baseline')


In [58]:
# ckpt_dir = os.path.join('..', 'runs', 'experiment_att')
# ckpt_path = os.path.join(ckpt_dir, 'experiment_final_model.pt') 
# ckpt = torch.load(ckpt_path)

# ckpt_model = selfattn_3DCNN(in_num_ch=1)
# ckpt_model.load_state_dict(ckpt)

# plot_dir = os.path.join('..', 'net_visualization', 'experiment_att')


In [32]:
# ckpt_dir = os.path.join('..', 'runs', 'experiment_res')
# ckpt_path = os.path.join(ckpt_dir, 'experimentres_final_model.pt') 
# ckpt = torch.load(ckpt_path)

# ckpt_model = resattn_3DCNN(in_num_ch=1)
# ckpt_model.load_state_dict(ckpt)

# plot_dir = os.path.join('..', 'net_visualization', 'experiment_res')


<All keys matched successfully>

### 3. Saliency map

In [62]:
USE_GPU = True

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
#     dtype = torch.cuda.FloatTensor
else:
    device = torch.device('cpu')
print(device)

cuda


In [63]:
gc.collect()

61206

In [ ]:
full_sal_df = pd.DataFrame()

with tqdm(image_loader, unit="batch") as tepoch:
    for t, (x, y, z) in enumerate(tepoch): # For each batch
        gc.collect()
            
        tepoch.set_description("Batch %d" % t)
        
        # Some default constants
        N = y.shape[0]
        ICHD = y.shape[1]

        # Compute saliency and rank of slices
        saliency = compute_saliency_maps(x, y, ckpt_model, device) # (ICH_types, N, D, H, W)
        saliency_rank = rank_saliency_slices(saliency) #(ICH_types, N, D)
        
        # Compute whether prediction was correct or not
        ckpt_model = ckpt_model.to(device)
        ckpt_model.eval()
        
        preds = (torch.sigmoid(ckpt_model(x.to(device, dtype=torch.float))) >= 0.5).long()
        corr_bool = (y == preds.cpu()).long() # N X 6

        # Save plot of saliency map in each patient directory
        # Only do up to 10 batches for plotting (40 patients)
        if t < 10:
            for p_index in range(N):
                patient_id = z[p_index].item()
                patient_ichs_ind = y[p_index, :].nonzero(as_tuple=True)[0] # Only ICH nums which patients have
                if len(patient_ichs_ind) == 0: # Skip patient if patient doesn't have any ICH
                    continue 
                patient_corr = corr_bool[p_index, :]

                # Make plot directory for patient
                dir_nm = '_'.join([patient_id, 
                                   'numICHs', str(len(patient_ichs_ind)), 
                                   'numCorr', str(patient_corr[patient_ichs_ind].sum().item())
                                  ])
                patient_dir = os.path.join(plot_dir, dir_nm)
                if not os.path.exists(patient_dir):
                    os.mkdir(patient_dir)

                for ich_num in patient_ichs_ind.cpu().numpy():
                    sal_plot = plot_saliency_maps(x, saliency, ich_num=ich_num, patient_id=p_index, 
                                                  d_range=np.arange(0,40), num_rows = 2*8, num_cols = 5)
                    sal_plot.savefig(os.path.join(patient_dir, '_'.join(['ICHNum', str(ich_num), 'Corr', str(patient_corr[ich_num].item())])))
                    # Clear the current axes.
                    plt.cla() 
                    # Clear the current figure.
                    plt.clf() 
                    sal_plot.clear()
                    plt.close('all')
                    del sal_plot
                
        # Convert to pandas df to save general information (patient ID, num of ICH class, pred corr or not (per num ICH class), sal rank per ICH class)
        save_df = pd.DataFrame(saliency_rank.detach().cpu().numpy().reshape(ICHD*N, -1)) # (ICH_types*N, D)
        save_df.columns = ['rank_' + str(rk) for rk in save_df.columns]

        save_df['patient_id'] = z.repeat(ICHD)
        save_df['ICH_num'] = np.tile(np.arange(0, ICHD), N)
        save_df['ICH_true'] = y.numpy().astype(int).reshape(-1)
        save_df['ICH_pred'] = preds.detach().cpu().numpy().astype(int).reshape(-1)
        save_df['corr_pred'] = corr_bool.detach().numpy().reshape(-1)
        save_df = save_df[save_df.columns.tolist()[-5:] + save_df.columns.tolist()[:-5]]

        full_sal_df = pd.concat([full_sal_df, save_df], axis=0)
        
        # Delete to free up memory?
        del saliency
        del saliency_rank
        del x
        
        print('CUDA:', torch.cuda.memory_allocated())
        print('RAM:', psutil.virtual_memory().percent)

full_sal_df.shape


Batch 0:   0%|          | 1/384 [01:40<10:39:10, 100.13s/batch]

CUDA: 318632960
RAM: 39.9


Batch 1:   1%|          | 2/384 [03:28<11:07:43, 104.88s/batch]

CUDA: 318632960
RAM: 42.9


Batch 2:   1%|          | 3/384 [04:21<8:34:40, 81.05s/batch]  

...

Batch 333:  87%|████████▋ | 334/384 [33:52<03:37,  4.34s/batch]

CUDA: 318632960
RAM: 59.5


Batch 334:  87%|████████▋ | 335/384 [33:57<03:33,  4.37s/batch]

CUDA: 318632960
RAM: 59.5


Batch 335:  87%|████████▋ | 335/384 [33:58<03:33,  4.37s/batch]

In [40]:
full_sal_df.to_csv(os.path.join(plot_dir, 'saliency_statistics.csv'), index=False)

In [41]:
full_sal_df.shape

(9216, 45)

### Plots only for top 10 slices for t=15 batches:

In [65]:
with tqdm(image_loader, unit="batch") as tepoch:
    for t, (x, y, z) in enumerate(tepoch): # For each batch
        if t > 15:
            break
        gc.collect()
            
        tepoch.set_description("Batch %d" % t)
        
        # Some default constants
        N = y.shape[0]
        ICHD = y.shape[1]

        # Compute saliency and rank of slices
        saliency = compute_saliency_maps(x, y, ckpt_model, device) # (ICH_types, N, D, H, W)
        saliency_rank = rank_saliency_slices(saliency) #(ICH_types, N, D)
        
        # Compute whether prediction was correct or not
        ckpt_model = ckpt_model.to(device)
        ckpt_model.eval()
        
        preds = (torch.sigmoid(ckpt_model(x.to(device, dtype=torch.float))) >= 0.5).long()
        corr_bool = (y == preds.cpu()).long() # N X 6

        # Save plot of saliency map in each patient directory
        # Only do up to 10 batches for plotting (40 patients)
        for p_index in range(N):
            patient_id = z[p_index].item()
            patient_ichs_ind = y[p_index, :].nonzero(as_tuple=True)[0] # Only ICH nums which patients have
            if len(patient_ichs_ind) == 0: # Skip patient if patient doesn't have any ICH
                continue 
            patient_corr = corr_bool[p_index, :]

            # Make plot directory for patient
            dir_nm = '_'.join([patient_id, 
                               'numICHs', str(len(patient_ichs_ind)), 
                               'numCorr', str(patient_corr[patient_ichs_ind].sum().item())
                              ])
            patient_dir = os.path.join(plot_dir, 'top_10', dir_nm)
            if not os.path.exists(patient_dir):
                os.mkdir(patient_dir)

            for ich_num in patient_ichs_ind.cpu().numpy():
                sal_plot = plot_saliency_maps(x, saliency, ich_num=ich_num, patient_id=p_index, 
                                              d_range=saliency_rank[ich_num, p_index, :10].detach().cpu().numpy(),
                                              num_rows=4, num_cols=5
                                             ) # only top 10 slices
                sal_plot.savefig(os.path.join(patient_dir, '_'.join(['ICHNum', str(ich_num), 'Corr', str(patient_corr[ich_num].item())])))
                # Clear the current axes.
                plt.cla() 
                # Clear the current figure.
                plt.clf() 
                sal_plot.clear()
                plt.close('all')
                del sal_plot

        # Delete to free up memory?
        del saliency
        del saliency_rank
        del x
        
        print('CUDA:', torch.cuda.memory_allocated())
        print('RAM:', psutil.virtual_memory().percent)


Batch 0:   0%|          | 1/384 [00:29<3:10:09, 29.79s/batch]

CUDA: 3997184
RAM: 39.2


Batch 1:   1%|          | 2/384 [00:47<2:23:28, 22.53s/batch]

CUDA: 3997184
RAM: 39.8


Batch 2:   1%|          | 3/384 [01:09<2:23:04, 22.53s/batch]

...

Batch 14:   4%|▍         | 15/384 [04:32<1:55:32, 18.79s/batch]

CUDA: 3997184
RAM: 39.8


Batch 15:   4%|▍         | 16/384 [04:56<2:06:17, 20.59s/batch]

CUDA: 3997184
RAM: 40.4


Batch 15:   4%|▍         | 16/384 [04:58<1:54:36, 18.69s/batch]


In [ ]:
# # Getting which patients have non ICH types
# patient_keys, nonzero_dims = y.nonzero(as_tuple=True)
# patient_keys = list(patient_keys.numpy())
# nonzero_dims = list(nonzero_dims.numpy())

# nonzero_dict = {}
# for i in range(len(patient_keys)):
#     nonzero_dict[patient_keys[i]] = nonzero_dict.get(patient_keys[i], []) + [nonzero_dims[i]]
# nonzero_dict